In [15]:
import requests
import re
import string
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords

In [16]:
# identify relevant IDs for Seeking Alpha API call

transcript_ids = ['4440497',
    '4432679',
    '4421803',
    '4410515',
    '4401190',
    '4392592',
    '4380137',
    '4373306',
    '4359774',
    '4352008',
    '4350619',
    '4339002',
    '4318472',
    '4312250',
    '4311715',
    '4310316',
    '4298035',
    '4291056',
    '4276888',
    '4269934',
    '4268332',
    '4264388',
    '4256114',
    '4235146',
    '4224895',
    '4213638',
    '4189681',
    '4165441',
    '4139579',
    '4116043',
    '4090470',
    '4065267',
    '4039167',
    '4015489',
    '3991324',
    '3968483',
    '3841096',
    '3591246',
    '3519276',
    '3348045',
    '3092676',
    '2851956',
    '2576895',
    '2327655',
    '2159723',
    '1958491',
    '1887121',
    '1875561',
    '1850701',
    '1758792',
    '1685532',
    '1674062',
    '1562752',
    '1492652',
    '1360101',
    '1257841',
    '1231171',
    '1217591',
    '1190771',
    '1126151',
    '1056141',
    '940131',
    '861271',
    '741461',
    '652031',
    '520021',
    '421771',
    '321469',
    '312799',
    '310047',
    '301745',
    '292596',
    '281666',
    '264164',
    '248315',
    '232213',
    '215253',
    '200961'
    ]

In [17]:
# pull transcripts and publish date from Seeking Alpha API

text_list = []


for transcript in transcript_ids:
    url = 'https://seeking-alpha.p.rapidapi.com/transcripts/get-details'

    id_string = {'id':transcript}

    id_headers = {
        'x-rapidapi-host': 'seeking-alpha.p.rapidapi.com',
        'x-rapidapi-key': '8839ca4bc9msh12a5fac3140b7b0p1cc9f7jsn6ae10955f355'
        }

    response = requests.request("GET", url, headers=id_headers, params=id_string)
    
    pub_date = response.json()['data']['attributes']['publishOn']
    
    my_text = response.json()['data']['attributes']['content']
    transcript = BeautifulSoup(my_text)
    raw_text = transcript.get_text()
    
    text_list.append([pub_date, raw_text])

In [18]:
# split API pull into Q&A versus prepared remarks

new_text_list = []

for item in text_list:
    if 'Question-and-Answer' in item[1]:
        x = item[1].split('Question-and-Answer')

        new_text_list.append([item[0], 'Prepared remarks',x[0]])
        new_text_list.append([item[0], 'Q&A',x[1]])
        
        
df = pd.DataFrame(new_text_list, columns=['date', 'section', 'content'])

In [19]:
# convert date to datetime

gen_dates = lambda x: x[:10]
df.date = pd.to_datetime(df.date.apply(gen_dates))

In [24]:
df

,date,section,content
0,2021-07-21,Prepared remarks,Texas Instruments Incorporated (NASDAQ:TXN) Q2...
1,2021-07-21,Q&A,Session\nOperator\nThank you. [Operator Instr...
2,2021-06-02,Prepared remarks,Texas Instruments Incorporated (NASDAQ:TXN) Be...
3,2021-06-02,Q&A,Session\nQ - \n
4,2021-04-27,Prepared remarks,Texas Instruments Incorporated. (NASDAQ:TXN) Q...
...,...,...,...
147,2010-10-26,Q&A,Session\nOperator\n[Operator Instructions] Ou...
148,2010-07-20,Prepared remarks,Texas Instruments Incorporated (NASDAQ:TXN) Q2...
149,2010-07-20,Q&A,Session\nOperator\nThank you. (Operator instr...
150,2010-04-27,Prepared remarks,Texas Instruments Incorporated (NASDAQ:TXN) Q1...


In [22]:
df.to_csv('transcript_df.csv', index=False)